In [1]:
import crystal_toolkit

In [2]:
from crystal_toolkit.components import structure
from pymatgen.core.structure import Structure
from crystal_toolkit.components.structure import StructureMoleculeComponent

from dash import Dash, html

In [5]:
structure=Structure.from_file('/Users/elena.patyukova/Documents/cif_files_ICSD/cifs_binary_Mg/YourCustomFileName_CollCode113.cif')

In [6]:
structure_component = StructureMoleculeComponent(structure, id="structure")

In [9]:
for i in structure_component.available_bonding_strategies:
    print(i)

VoronoiNN
JmolNN
MinimumDistanceNN
OpenBabelNN
CovalentBondNN
MinimumOKeeffeNN
MinimumVIRENN
BrunnerNNReciprocal
BrunnerNNRelative
BrunnerNNReal
EconNN
CrystalNN
CutOffDictNN
Critic2NN


In [18]:
app=Dash()

In [15]:
structure_component.generate_callbacks(app)

TypeError: StructureMoleculeComponent.generate_callbacks() missing 1 required positional argument: 'cache'

In [17]:
html.structure_component.layout())

dash.html.Div.Div

In [23]:
app.layout=html.Div(children=structure_component.layout(),id='stuff')

In [20]:
app.layout

Div(children=Div([CrystalToolkitScene(children=[Div(children=[Label(children='Change unit cell:', className='mpc-label'), Div(children=Dropdown(options=[{'label': 'Input cell', 'value': 'input'}, {'label': 'Primitive cell', 'value': 'primitive'}, {'label': 'Conventional cell', 'value': 'conventional'}, {'label': 'Reduced cell (Niggli)', 'value': 'reduced_niggli'}, {'label': 'Reduced cell (LLL)', 'value': 'reduced_lll'}], value='input', clearable=False, id='CTstructure_unit-cell-choice', persistence=True, persistence_type='local'), className='mpc-control'), Div([Label(children='Change bonding algorithm: ', className='mpc-label'), Dropdown(options=[{'label': 'CrystalNN', 'value': 'CrystalNN'}, {'label': 'Custom Bonds', 'value': 'CutOffDictNN'}, {'label': 'Jmol Bonding', 'value': 'JmolNN'}, {'label': 'Minimum Distance (10% tolerance)', 'value': 'MinimumDistanceNN'}, {'label': "O'Keeffe's Algorithm", 'value': 'MinimumOKeeffeNN'}, {'label': "Hoppe's ECoN Algorithm", 'value': 'EconNN'}, {'la

In [3]:
import numpy as np
import pandas as pd
from pymatgen.core.composition import Composition
from pymatgen.core.structure import Structure
from dotenv import load_dotenv
import os
import json
from mp_api.client import MPRester

load_dotenv()

True

## MP structure search

In [2]:
mp_api_key = os.environ.get('MP_API_KEY')

In [11]:
formula='Na2Cl2'

In [12]:
formula='Na2Cl2'
composition=Composition(formula)
formula,_=composition.get_reduced_formula_and_factor()

In [17]:
with MPRester(mp_api_key) as mpr:
    docs = mpr.materials.summary.search(
            formula=formula, is_stable="True"
            )

Retrieving SummaryDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

In [27]:
lowest_energy_index=0
lowest_energy=docs[0].energy_per_atom
if(len(docs)>1):
    for i in range(1,len(docs)):
        energy=docs[i].energy_per_atom
        if(energy<lowest_energy):
            lowest_energy=energy
            lowest_energy_index=i
mp_structure=docs[lowest_energy_index].structure

In [28]:
mp_structure

Structure Summary
Lattice
    abc : 3.951402922131455 3.951402172480169 3.951402
 angles : 59.99998470065274 59.99999097646918 59.99999737523848
 volume : 43.625325465502776
      A : 3.422015 0.0 1.975702
      B : 1.140671 3.226306 1.975702
      C : 0.0 0.0 3.951402
    pbc : True True True
PeriodicSite: Na (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Cl (2.281, 1.613, 3.951) [0.5, 0.5, 0.5]

## Jarvis Structure

In [29]:
from jarvis.db.figshare import data

In [30]:
dft_3d = data('dft_3d')
df=pd.DataFrame(dft_3d)

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.


In [31]:
df.columns

Index(['jid', 'spg_number', 'spg_symbol', 'formula',
       'formation_energy_peratom', 'func', 'optb88vdw_bandgap', 'atoms',
       'slme', 'magmom_oszicar', 'spillage', 'elastic_tensor',
       'effective_masses_300K', 'kpoint_length_unit', 'maxdiff_mesh',
       'maxdiff_bz', 'encut', 'optb88vdw_total_energy', 'epsx', 'epsy', 'epsz',
       'mepsx', 'mepsy', 'mepsz', 'modes', 'magmom_outcar', 'max_efg',
       'avg_elec_mass', 'avg_hole_mass', 'icsd', 'dfpt_piezo_max_eij',
       'dfpt_piezo_max_dij', 'dfpt_piezo_max_dielectric',
       'dfpt_piezo_max_dielectric_electronic',
       'dfpt_piezo_max_dielectric_ionic', 'max_ir_mode', 'min_ir_mode',
       'n-Seebeck', 'p-Seebeck', 'n-powerfact', 'p-powerfact', 'ncond',
       'pcond', 'nkappa', 'pkappa', 'ehull', 'Tc_supercon', 'dimensionality',
       'efg', 'xml_data_link', 'typ', 'exfoliation_energy', 'spg', 'crys',
       'density', 'poisson', 'raw_files', 'nat', 'bulk_modulus_kv',
       'shear_modulus_gv', 'mbj_bandgap', 'hse_ga

In [35]:
df['spg_symbol']

0        P4/nmm
1         Pm-3m
2         I-4m2
3        P2_1/c
4         P-3m1
          ...  
75988    I4/mmm
75989     F-43m
75990    I4/mmm
75991    I4/mmm
75992    I4/mmm
Name: spg_symbol, Length: 75993, dtype: object

In [33]:
df.loc[df['formula']==formula]

,jid,spg_number,spg_symbol,formula,formation_energy_peratom,func,optb88vdw_bandgap,atoms,slme,magmom_oszicar,...,density,poisson,raw_files,nat,bulk_modulus_kv,shear_modulus_gv,mbj_bandgap,hse_gap,reference,search
24665,JVASP-8555,221,Pm-3m,NaCl,-1.78722,OptB88vdW,4.343,"{'lattice_mat': [[3.458077088999604, -0.0, 0.0...",0.0,0.0,...,2.347,0.39,"[FD-ELAST,JVASP-8555.zip,https://ndownloader.f...",2,29.97,16.24,7.504,na,mp-22851,-Cl-Na
33057,JVASP-23862,225,Fm-3m,NaCl,-1.93651,OptB88vdW,5.321,"{'lattice_mat': [[3.4360963088789007, -2.88957...",0.0,na,...,2.197,0.25,"[FD-ELAST,JVASP-23862.zip,https://ndownloader....",2,27.67,17.8,8.251,na,mp-22862,-Cl-Na
61329,JVASP-135495,187,P-6m2,NaCl,-1.87302,OptB88vdW,4.578,"{'lattice_mat': [[5.146723890653078, -0.379435...",na,0.0,...,2.151,0.29,[],2,26.86,14.47,na,na,JQE-635098,-Cl-Na
61546,JVASP-135501,123,P4/mmm,NaCl,-1.72431,OptB88vdW,4.161,"{'lattice_mat': [[3.8161365742822784, -0.0, 0....",na,0.0,...,1.434,0.62,[],2,14.38,9.23,na,na,JQE-635140,-Cl-Na


In [36]:
da=df.loc[df['formula']==formula]
da.reset_index(inplace=True,drop=True)
lowest_energy_index=0
lowest_energy=da.iloc[0]['formation_energy_peratom']
if(len(da)>1):
    for i in range(1,len(da)):
        energy=da.iloc[i]['formation_energy_peratom']
        if(energy<lowest_energy):
            lowest_energy=energy
            lowest_energy_index=i
atoms=da.iloc[lowest_energy_index]['atoms']
structure=Structure(lattice=atoms['lattice_mat'],species=atoms['elements'],coords=atoms['coords'])

In [39]:
# Downloading structure from MC3D

In [ ]:
index=

In [43]:
len(os.listdir('/Users/elena.patyukova/Documents/github/ML_for_Me_prediction/Kspacing/data/MC3D-data-PBE'))

15217

In [45]:
structure.to('test.cif')

"# generated using pymatgen\ndata_NaCl\n_symmetry_space_group_name_H-M   'P 1'\n_cell_length_a   3.96766254\n_cell_length_b   3.96766288\n_cell_length_c   3.96766243\n_cell_angle_alpha   59.99999574\n_cell_angle_beta   59.99999297\n_cell_angle_gamma   59.99999809\n_symmetry_Int_Tables_number   1\n_chemical_formula_structural   NaCl\n_chemical_formula_sum   'Na1 Cl1'\n_cell_volume   44.16611160\n_cell_formula_units_Z   1\nloop_\n _symmetry_equiv_pos_site_id\n _symmetry_equiv_pos_as_xyz\n  1  'x, y, z'\nloop_\n _atom_site_type_symbol\n _atom_site_label\n _atom_site_symmetry_multiplicity\n _atom_site_fract_x\n _atom_site_fract_y\n _atom_site_fract_z\n _atom_site_occupancy\n  Na  Na0  1  0.00000000  0.00000000  0.00000000  1\n  Cl  Cl1  1  2.29073500  1.61979000  3.96766000  1\n"

In [48]:
from pymatgen.io.cif import CifWriter

In [50]:
write_cif=CifWriter(structure)
write_cif.write_file('test.cif')

In [52]:
df=pd.read_json('/Users/elena.patyukova/Documents/github/qe-input/src/qe_input/mc3d_structures/mc3d_filtered_entries_pbe-v1_2025-01-16-01-09-20.json')

In [54]:
df.loc[df['formula']=='Ti']

,formula,formula_hill,spacegroup_number,source_db,source_db_id,is_theoretical,is_high_pressure,is_high_temperature,num_elements,id,num_atoms,bravais_lat,spacegroup_int,details_link,download_cif,total_magnetization,absolute_magnetization
5560,Ti,Ti2,194,MPDS,S1020386,no,no,no,1,mc3d-21616/pbe-v1,2,hP,P6_3/mmc,https://mc3d.materialscloud.org/#/details/mc3d...,https://aiida.materialscloud.org/mc3d/api/v4/n...,NaN,NaN
6424,Ti,Ti,139,ICSD,671047,yes,no,no,1,mc3d-23412/pbe-v1,1,tI,I4/mmm,https://mc3d.materialscloud.org/#/details/mc3d...,https://aiida.materialscloud.org/mc3d/api/v4/n...,NaN,NaN
9470,Ti,Ti3,191,MPDS,S1254092,no,no,no,1,mc3d-29736/pbe-v1,3,hP,P6/mmm,https://mc3d.materialscloud.org/#/details/mc3d...,https://aiida.materialscloud.org/mc3d/api/v4/n...,NaN,NaN
18619,Ti,Ti,229,MPDS,S534323,no,no,yes,1,mc3d-48953/pbe-v1,1,cI,Im-3m,https://mc3d.materialscloud.org/#/details/mc3d...,https://aiida.materialscloud.org/mc3d/api/v4/n...,NaN,NaN
28082,Ti,Ti,225,ICSD,168322,yes,no,no,1,mc3d-68640/pbe-v1,1,cF,Fm-3m,https://mc3d.materialscloud.org/#/details/mc3d...,https://aiida.materialscloud.org/mc3d/api/v4/n...,NaN,NaN
33594,Ti,Ti2,191,COD,9011600,no,NaN,NaN,1,mc3d-8175/pbe-v1,2,hP,P6/mmm,https://mc3d.materialscloud.org/#/details/mc3d...,https://aiida.materialscloud.org/mc3d/api/v4/n...,NaN,NaN


In [62]:
def mc3d_structure_lookup(formula):
    df=pd.read_json('mc3d_structures/mc3d_filtered_entries_pbe-v1_2025-01-16-01-09-20.json')
    formula=Composition(formula).hill_formula

    da=df.loc[df['formula_hill']==formula]
    da.reset_index(inplace=True,drop=True)
    lowest_energy_index=0
    lowest_energy=da['total_energy'].values[0]

    if(len(da)>1):
        for i in range(1,len(da)):
            energy=da['total_energy'].values[i]
            if(energy < lowest_energy):
                lowest_energy=energy
                lowest_energy_index=i
    ID=da['id'].values[lowest_energy_index]   
    structure_file='mc3d_structures/mc3d-pbe-cifs/'+ID[:-7]+'-pbe.cif' 
    structure=Structure.from_file(structure_file)
    return structure

In [64]:
mc3d_structure_lookup('PO')

IndexError: index 0 is out of bounds for axis 0 with size 0

In [1]:
import requests
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import re
import copy
from bs4 import BeautifulSoup

In [111]:
formula='Ti O2'

SERVER="http://aflow.org"
API="/API/aflux/v1.0/?"
MATCHBOOK="nspecies(2),compound,data_source,Egap(0*,*20),Egap_type,energy_cutoff,pressure,geometry,catalog,positions_cartesian,kpoints_bands_nkpts,kpoints_relax,kpoints_static,spacegroup_orig,spacegroup_relax,dft_type,aflow_version,aflowlib_date,aflowlib_version,auid,code"
DIRECTIVES="$paging(0)"
SUMMONS=MATCHBOOK+","+DIRECTIVES
# Make the GET request
response = requests.get('SERVER+API+SUMMONS')


In [129]:
aflow_url = "http://aflow.org/API/aflux/v1.0/?"

# Create the query to filter for Fe2O3
# Add required fields, such as composition, structure, and formation energy
query = (
    f"composition(Fe2O3),auid,composition,structure,enthalpy_formation_atom,"
    f"geometry,spacegroup,prototype"
)

# Complete API URL
full_url = f"{aflow_url}{query}"

In [184]:
full_url='http://aflow.org/API/aflux/v1.0/?species(Ti),compound,data_source,energy_cutoff,pressure,geometry,positions_cartesian,kpoints_bands_nkpts,kpoints_relax,kpoints_static,dft_type'

In [185]:
response = requests.get(full_url)

In [186]:
html_content = response.content
soup = BeautifulSoup(html_content, 'html.parser')
content=json.loads(soup.text)

In [187]:
formula='Ti'

In [188]:
indexes=[]
for i,entry in enumerate(content):
    form=Composition(entry['compound']).hill_formula
    formula=Composition(formula).hill_formula
    if(formula==form):
        indexes.append[i]

In [189]:
indexes

[]

In [125]:
species=[]
coords=[]
for line in content['data'][0]['sites']:
    el, merged_coord = line.split(' @ ')
    species.append(el)
    x,y,z=merged_coord.split(' ')
    coords.append([float(x),float(y),float(z)])
Structure(lattice=content['data'][0]['unit_cell'],species=species,coords=coords)

In [126]:
species

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'Ti', 'Ti', 'Ti', 'Ti']

Structure Summary
Lattice
    abc : 7.311786297681497 7.311786297681497 7.311786297681497
 angles : 92.3472707915692 92.3472707915692 156.6481000859169
 volume : 151.73943694928326
      A : -5.06323081 5.06323081 1.479731865
      B : 5.06323081 -5.06323081 1.479731865
      C : 5.06323081 5.06323081 -1.479731865
    pbc : True True True
PeriodicSite: O (-2.997, 3.494, 1.48) [0.845, 0.204, 0.0491]
PeriodicSite: O (-0.4253, 3.402, 1.48) [0.836, 0.458, 0.294]
PeriodicSite: O (1.57, 2.066, 4.523e-17) [0.204, 0.155, 0.359]
PeriodicSite: O (3.397, 0.4202, 1.481) [0.542, 0.836, 0.377]
PeriodicSite: O (1.666, 4.643, -0.00148) [0.458, 0.164, 0.623]
PeriodicSite: O (3.494, 2.997, 1.48) [0.796, 0.845, 0.641]
PeriodicSite: O (5.489, 1.661, 1.036e-16) [0.164, 0.542, 0.706]
PeriodicSite: O (8.061, 1.57, 1.09e-16) [0.155, 0.796, 0.951]
PeriodicSite: Ti (3.321, -1.489, 1.48) [0.353, 0.828, 0.181]
PeriodicSite: Ti (1.489, 3.321, 1.48) [0.828, 0.647, 0.475]
PeriodicSite: Ti (3.575, 1.742, -8.282e-17) 

In [95]:
import qmpy_rester as qr

## Return list of data
with qr.QMPYRester() as q:
    kwargs = {
        'composition': 'Ti O2',      # composition include (Fe OR Mn) AND O
        }
    list_of_data = q.get_oqmd_phases(**kwargs)

Your filters are:
    composition=Ti O2
Proceed? [Y/n]:y


In [98]:
list_of_data

{'links': {'next': 'http://oqmd.org/oqmdapi/formationenergy?composition=Ti+O2&limit=50&offset=50',
  'previous': None,
  'base_url': {'href': 'https://oqmd.org/oqmdapi',
   'meta': {'_oqmd_version': '1.0'}}},
 'resource': {},
 'data': [{'name': 'TiO2',
   'entry_id': 2475,
   'calculation_id': 15978,
   'icsd_id': 9161,
   'formationenergy_id': 4071720,
   'duplicate_entry_id': 2475,
   'composition': 'O2 Ti1',
   'composition_generic': 'AB2',
   'prototype': 'TiO2(tP6)',
   'spacegroup': 'P42/mnm',
   'volume': 62.7514,
   'ntypes': 2,
   'natoms': 6,
   'unit_cell': [[4.606484, 0.0, 0.0],
    [0.0, 4.606484, 0.0],
    [0.0, 0.0, 2.957225]],
   'sites': ['O @ 0.304 0.304 0',
    'O @ 0.696 0.696 0',
    'O @ 0.804 0.196 0.5',
    'O @ 0.196 0.804 0.5',
    'Ti @ 0 0 0',
    'Ti @ 0.5 0.5 0.5'],
   'band_gap': 1.745,
   'delta_e': -3.21575255518126,
   'stability': 0.02688833666666035,
   'fit': 'standard',
   'calculation_label': 'static'},
  {'name': 'TiO2',
   'entry_id': 2575,
   '

In [ ]:
response = json.loads(urlopen(SERVER+API+SUMMONS).read().decode("utf-8"))

In [190]:
import numpy as np
import pandas as pd
import math
import json
import re
import os
import shutil
from sklearn.model_selection import train_test_split
from jarvis.db.figshare import data
import matplotlib.pyplot as plt
from pymatgen.core.composition import Composition

In [36]:
dft_3d = data('dft_3d')
df=pd.DataFrame(dft_3d)

Obtaining 3D dataset 76k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815699
Loading the zipfile...
Loading completed.


In [40]:
da.loc[da['formula']=='NiO']['atoms'].values[2]

{'lattice_mat': [[0.8473128044042595, 2.3965619010322654, 1.4675883828292455],
  [-2.541563597828537, 0.0002639701199552, 4.402117632145438],
  [1.6946246381184444, -2.3965619058366983, -1.2975563e-09]],
 'coords': [[0.0, 0.0, 0.0],
  [0.00018499999999999073, 0.00012999999999996348, 2.9348549999999998],
  [0.8474045300193643, -1.1982135645548428, 1.467423983897084],
  [-0.8470345300193648, 1.198473564554843, 4.4022860161029165]],
 'elements': ['Ni', 'Ni', 'O', 'O'],
 'abc': [2.935175, 5.083128, 2.935172],
 'angles': [106.7811, 120.0001, 73.2188],
 'cartesian': True,
 'props': ['', '', '', '']}

In [37]:
da=df[['jid','formula','formation_energy_peratom','atoms', 'spg_symbol']]

In [38]:
da.to_pickle('Jarvis.pkl')

In [209]:
from zipfile import ZipFile

In [ ]:
with myzip.open('eggs.txt') as myfile:
        print(myfile.read())

In [215]:
with ZipFile('Jarvis.pkl.zip') as myzip:
    with myzip.open('Jarvis.pkl') as myfile:
        dc=pd.read_pickle(myzip)

AttributeError: 'ZipFile' object has no attribute 'seek'

In [203]:
da=pd.read_pickle('jarvis.pkl')

In [204]:
da

,formula,formation_energy_peratom,atoms
0,TiCuSiAs,-0.42762,"{'lattice_mat': [[3.566933224304235, 0.0, -0.0..."
1,DyB6,-0.41596,"{'lattice_mat': [[4.089078911208881, 0.0, 0.0]..."
2,Be2OsRu,0.04847,"{'lattice_mat': [[-1.833590720595598, 1.833590..."
3,KBi,-0.44140,"{'lattice_mat': [[7.2963518353359165, 0.0, 0.0..."
4,VSe2,-0.71026,"{'lattice_mat': [[1.6777483798834445, -2.90594..."
...,...,...,...
75988,AcRh2Pb2,-0.30652,"{'lattice_mat': [[-2.374509402119681, 2.374509..."
75989,PrTlZn,-0.34112,"{'lattice_mat': [[-0.0, 3.4210598347774503, 3...."
75990,BaIn2Bi2,-0.39352,"{'lattice_mat': [[4.082347574975881, -4.076131..."
75991,TmSi2Tc2,-0.54853,"{'lattice_mat': [[2.90400678672412, -2.9037689..."


In [25]:
from pymatgen.core.structure import Structure

In [26]:
formula='TiO2'

In [27]:
df=pd.read_pickle('/Users/elena.patyukova/Documents/github/qe-input/src/qe_input/Jarvis.pkl')
da=df.loc[df['formula']==formula]
da.reset_index(inplace=True,drop=True)

In [28]:
da

,jid,formula,formation_energy_peratom,atoms
0,JVASP-42549,TiO2,-3.16279,"{'lattice_mat': [[3.023060547465333, -0.000961..."
1,JVASP-90247,TiO2,-2.98834,"{'lattice_mat': [[0.0, 0.0, -3.112773633009819..."
2,JVASP-17258,TiO2,-0.41758,"{'lattice_mat': [[1.6953034703885244, -2.93635..."
3,JVASP-314,TiO2,-3.24470,"{'lattice_mat': [[0.0, 3.8072490526345937, -0...."
4,JVASP-305,TiO2,-0.41758,"{'lattice_mat': [[1.6950443309663075, -2.93590..."
5,JVASP-105583,TiO2,-3.08953,"{'lattice_mat': [[2.748917895721392, -0.072228..."
6,JVASP-18983,TiO2,-3.24726,"{'lattice_mat': [[5.171752472265345, -0.0, 0.0..."
7,JVASP-22573,TiO2,-3.24261,"{'lattice_mat': [[3.7561031242041443, -7.72820..."
8,JVASP-8810,TiO2,-3.24407,"{'lattice_mat': [[3.7979943773480436, -0.00970..."
9,JVASP-308,TiO2,-3.08044,"{'lattice_mat': [[2.749433611265347, -0.075525..."


In [18]:
formulas=[]
energy=[]
groups=[]
natoms=[]
abc=[]
angles=[]
jid=[]
for i in range(len(da)):
    atoms=da['atoms'].values[i]
    structure=Structure(lattice=atoms['lattice_mat'],species=atoms['elements'],coords=atoms['coords'])
    groups.append(structure.get_space_group_info(symprec = 0.01, angle_tolerance = 5.0)[0])
    structure=structure.get_reduced_structure()
    ABC=[round(x,2) for x in structure.lattice.abc]
    Angles=[round(x,1) for x in structure.lattice.angles]
    natoms.append(structure.num_sites)
    abc.append(ABC)
    angles.append(Angles)
    energy.append(da['formation_energy_peratom'].values[i])
    formulas.append(structure.formula)

In [19]:
result=pd.DataFrame()
result['formula']=formulas
result['energy']=energy
result['sg']=groups
result['abc']=abc
result['angles']=angles
result=result.sort_values(by=['energy'])
result.reset_index(drop=True, inplace=True)

In [20]:
result=result.sort_values(by=['energy'])
result.reset_index(drop=True, inplace=True)
result

,formula,energy,sg,abc,angles
0,Ti4 O8,-3.26318,P1,"[3.8, 3.81, 9.61]","[90.0, 90.0, 90.0]"
1,Ti4 O8,-3.24965,P1,"[4.57, 4.93, 5.56]","[90.0, 90.0, 90.0]"
2,Ti4 O8,-3.24859,P1,"[5.5, 5.5, 5.5]","[106.9, 106.9, 114.6]"
3,Ti8 O16,-3.24726,P-1,"[5.17, 5.5, 9.25]","[90.0, 90.0, 90.0]"
4,Ti4 O8,-3.24470,P1,"[3.81, 3.81, 9.63]","[90.0, 90.0, 90.0]"
5,Ti2 O4,-3.24411,P-1,"[3.81, 3.81, 5.52]","[110.2, 110.2, 90.0]"
6,Ti2 O4,-3.24407,P-1,"[3.81, 3.81, 5.52]","[69.8, 69.8, 90.0]"
7,Ti4 O8,-3.24261,P-1,"[3.76, 6.42, 6.61]","[106.2, 90.0, 107.0]"
8,Ti4 O8,-3.24118,P1,"[5.5, 5.5, 5.5]","[107.0, 107.0, 114.6]"
9,Ti2 O4,-3.24104,Cm,"[2.97, 4.63, 4.63]","[90.0, 90.0, 90.0]"


In [265]:
formulas=da['formula'].values
form_energy

NameError: name 'form_energy' is not defined

In [237]:
atoms=da.iloc[2]['atoms']
structure=Structure(lattice=atoms['lattice_mat'],species=atoms['elements'],coords=atoms['coords'])

In [238]:
structure.get_space_group_info(symprec = 0.02, angle_tolerance = 5.0)

('Fm-3m', 225)

In [239]:
structure.get_space_group_info(symprec = 0.01, angle_tolerance = 5.0)

('Fm-3m', 225)

In [240]:
structure.to_primitive()

Structure Summary
Lattice
    abc : 2.889113885711722 2.889113885711722 2.889113885711722
 angles : 59.99999999999999 59.99999999999999 59.99999999999999
 volume : 17.052143822654166
      A : -1.250922833243062e-16 2.0429120202069746 2.0429120202069746
      B : 2.0429120202069746 0.0 2.0429120202069746
      C : 2.0429120202069746 2.0429120202069746 2.501845666486124e-16
    pbc : True True True
PeriodicSite: Ti (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]

In [256]:
structure=structure.get_reduced_structure()

In [248]:
structure.lattice.angles

(60.0000108737582, 60.00000604430371, 60.000002360630205)

In [235]:
    st.
    lowest_energy_index=0
    lowest_energy=da.iloc[0]['formation_energy_peratom']
    if(len(da)>1):
        for i in range(1,len(da)):
            energy=da.iloc[i]['formation_energy_peratom']
            if(energy<lowest_energy):
                lowest_energy=energy
                lowest_energy_index=i
    atoms=da.iloc[lowest_energy_index]['atoms']
    structure=Structure(lattice=atoms['lattice_mat'],species=atoms['elements'],coords=atoms['coords'])


SyntaxError: invalid syntax (1819360923.py, line 1)